# 시각화 : 지도 데이터 활용

In [1]:
import pandas as pd
from math import sin, cos, sqrt, atan2, radians


## folium 라이브러리 
Folium 은 ‘Open Street Map’과 같은 지도데이터에 ‘Leaflet.js’를 이용하여 위치정보를 시각화하기 위한 라이브러리다. 기본적으로 ‘GeoJSON’ 형식 또는 ‘topoJSON’ 형식으로 데이터를 지정하면, 오버레이를 통해 마커의 형태로 위치 정보를 지도상에 표현할 수 있다.

우리가 현실세계에서 지도상의 위치를 표시하기 위해 ‘위도(latitude)’와 ‘경도(longitude)’를 사용한다. ‘위도’는 적도를 기준으로 하여 남쪽으로 남극점까지 90°, 북쪽으로 북극점까지 90°로 나누어 표시한다(우리나라는 적도의 북쪽인 북위 34° ~ 38° 사이에 위치). ‘경도(longitude)는 런던 그리니치천문대를 지나는 본초자오선을 중심으로 동서로 나누어 동경 180°, 서경 180°로 분리한다(서울의 경우 동경 127°에 위치).
### 위도경도 찾는 방법
https://www.google.com/maps


In [2]:
#!pip install folium
import folium

### 데이터 로드

In [3]:
df = pd.read_csv('data/starbucks.csv')
df.head()

,coord,addr,name
0,"[37.50380028448786,126.77415434253784]",경기도 부천시 원미구 중동 1058-5,스타벅스 신중동역점
1,"[37.50541300944734,126.75262753922486]",경기도 부천시 원미구 상동 544-4,스타벅스 부천상동점
2,"[37.52949798932593,126.83197703906416]",서울특별시 양천구 신월동 205-50,스타벅스 신월IC DT점
3,"[37.54061619745844,126.83740476165849]",서울특별시 강서구 화곡동 1073-9,스타벅스 화곡동점
4,"[37.4979195975082,126.77705510477135]",경기도 부천시 원미구 중동 1131-5,스타벅스 부천중동점


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 911 entries, 0 to 910
Data columns (total 3 columns):
coord    911 non-null object
addr     911 non-null object
name     911 non-null object
dtypes: object(3)
memory usage: 21.4+ KB


### 지도에 위치 표시

In [5]:
'''
map_osm = folium.Map(location=[37.5491,126.98955],
                     tiles=tileset,
                     zoom_start=14,
                     attr='cartocdn')

'''

#map_osm = folium.Map(location=[37.5491,126.98955], zoom_start=14)

# 일산병원 위도 경도 : @37.645818,126.793028
org_lat = 37.645818
org_lng = 126.793028
map_osm = folium.Map(location=[org_lat, org_lng], zoom_start=15)

# Simple marker
#folium.Marker(lat, lng], popup='일산병원').add_to(map_osm)

# change color and icon
folium.Marker([org_lat, org_lng], popup='일산병원', icon=folium.Icon(color='green',icon='info-sign')).add_to(map_osm)

# Circle marker
folium.CircleMarker([org_lat, org_lng], radius=100,color='#3186cc',fill_color='#3186cc', popup='일산병원').add_to(map_osm)

# Interactive marker
# map_osm.add_child(folium.ClickForMarker(popup='여기는 어디일까요?'))

org_lng = radians(float(org_lng))
org_lat = radians(float(org_lat))
R = 6373.0

for idx, df_iter in df.iterrows():
    df_iter['coord'] = df_iter['coord'].replace('[', '')
    df_iter['coord'] = df_iter['coord'].replace(']', '')
    lat, lng = df_iter['coord'].split(',')
    
#    popup = folium.Popup(df_iter['name'])
    popup = df_iter['name']
  
    # 특정 거리에 위치한 지점 만 표시하기 위한 처리
    tg_lng = radians(float(lng))
    tg_lat = radians(float(lat))

    dlon = abs(org_lng - tg_lng)
    dlat = abs(org_lat - tg_lat)
    
    a = sin(dlat / 2)**2 + cos(org_lat) * cos(tg_lat) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))

    # km로 변경
    distance = R * c

    # 5km 거리에 있는 스타벅스 지점만 보여줌  
    if (distance <= 5) :
        # print("Result : ", distance , " : popup", popup )
        popup = popup + str("{0:5.1f}".format(distance) + "km")

        circle = folium.CircleMarker(location=[float(lat), float(lng)],
                                     radius=5,
                                     popup=popup,
                                     color='black',
                                     fill_color='#00FF00',
                                     fill_opacity=0.5)

        map_osm.add_child(circle)
    
map_osm

In [6]:
# map을 저장
map_osm.save('data/starbucks_data_map.html')

In [ ]:
help(folium.Icon)